# RAG using Meta AI Llama-3


<img src="./resources/rag_architecture.png" width=800px>

In [1]:
import nest_asyncio
from dotenv import load_dotenv
from IPython.display import Markdown, display

from llama_index.core import Settings
from llama_index.llms.ollama import Ollama
from llama_index.core import PromptTemplate
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import VectorStoreIndex, ServiceContext, SimpleDirectoryReader

from llama_index.vector_stores.qdrant import QdrantVectorStore
from llama_index.core import Settings
import qdrant_client

In [2]:
# allows nested access to the event loop
nest_asyncio.apply()

In [3]:
# add your documents in this directory, you can drag & drop
input_dir_path = '/teamspace/studios/this_studio/medical_documents'

In [4]:
collection_name="chat_with_docs"

client = qdrant_client.QdrantClient(
    host="localhost",
    port=6333
)

def create_index(documents):
    vector_store = QdrantVectorStore(client=client, collection_name=collection_name)
    storage_context = StorageContext.from_defaults(vector_store=vector_store)
    index = VectorStoreIndex.from_documents(
        documents,
        storage_context=storage_context,
    )
    return index

In [5]:

# setup llm & embedding model
llm=Ollama(model="llama3:8b-instruct-q4_1", request_timeout=120.0)
# embed_model = HuggingFaceEmbedding( model_name="Snowflake/snowflake-arctic-embed-m", trust_remote_code=True)
embed_model = HuggingFaceEmbedding( model_name="BAAI/bge-large-en-v1.5", trust_remote_code=True)

config.json:   0%|          | 0.00/779 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [7]:
# load data
loader = SimpleDirectoryReader(
            input_dir = input_dir_path,
            required_exts=[".pdf"],
            recursive=True
        )
docs = loader.load_data()

# Creating an index over loaded data
Settings.embed_model = embed_model
try:
    index = create_index(docs)
    print('Using Qdrant collection')
except:
    index = VectorStoreIndex.from_documents(docs, show_progress=True)

# Create the query engine, where we use a cohere reranker on the fetched nodes
Settings.llm = llm
query_engine = index.as_query_engine()

# ====== Customise prompt template ======
broad_prompt = '''
You are an expert clinical assistant AI. Your task is to help healthcare providers with diagnostic support based on patient symptoms, medical history, and local disease data.

- If the patient's symptoms match a known diagnosis based on your pre-existing knowledge or provided documents, suggest the diagnosis and treatment options.
- Otherwise, use the search tool to retrieve relevant medical information.
- In addition to diagnosis, suggest follow-up consultation questions for the healthcare provider to ask the patient to ensure a comprehensive diagnosis.
- Provide classifications of possible diseases based on the symptoms, including differential diagnoses (similar conditions to rule out).
- Ensure to ask further clarifying questions that the doctor should consider, such as lifestyle factors, prior medical history, or exposure to local disease trends, to refine the diagnosis.
- If the user does not provide all the necessary information (e.g., symptoms, medical history, or location), ask them to specify the missing fields.
'''
qa_prompt_tmpl =  PromptTemplate(broad_prompt)

query_engine.update_prompts(
    {"response_synthesizer:text_qa_template": qa_prompt_tmpl}
)

# Generate the response
# Generate the response with an example query
response = query_engine.query("What are the potential diagnoses for a patient with a cough, fever, and asthma?",)

# Print the generated response
print("Generated Response:", response)


Parsing nodes:   0%|          | 0/5229 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/2048 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/2048 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/2048 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/1673 [00:00<?, ?it/s]

Generated Response: I'm happy to assist with diagnostic support. Please provide me with the patient's symptoms, medical history, and local disease data. I'll do my best to suggest a diagnosis and treatment options based on the information provided.

Please note that I'm an AI-powered clinical assistant, and while I can provide valuable insights and suggestions, it's essential to remember that I am not a substitute for a qualified healthcare professional. A comprehensive diagnosis requires careful consideration of various factors, including medical history, physical examination findings, laboratory results, and other relevant information.

To get started, please provide the following information:

* Symptoms: Please describe the patient's symptoms in detail, including any relevant dates or duration of symptoms.
* Medical history: Please provide a brief summary of the patient's medical history, including any significant conditions, surgeries, allergies, or previous diagnoses.
* Local dis

In [8]:
display(Markdown(str(response)))

I'm happy to assist with diagnostic support. Please provide me with the patient's symptoms, medical history, and local disease data. I'll do my best to suggest a diagnosis and treatment options based on the information provided.

Please note that I'm an AI-powered clinical assistant, and while I can provide valuable insights and suggestions, it's essential to remember that I am not a substitute for a qualified healthcare professional. A comprehensive diagnosis requires careful consideration of various factors, including medical history, physical examination findings, laboratory results, and other relevant information.

To get started, please provide the following information:

* Symptoms: Please describe the patient's symptoms in detail, including any relevant dates or duration of symptoms.
* Medical history: Please provide a brief summary of the patient's medical history, including any significant conditions, surgeries, allergies, or previous diagnoses.
* Local disease data: If available, please share local disease trends, outbreaks, or seasonal patterns that may be relevant to the patient's symptoms.

Once I have this information, I'll do my best to:

1. Identify potential diagnoses based on the provided symptoms and medical history.
2. Suggest treatment options for each possible diagnosis.
3. Provide follow-up consultation questions for the healthcare provider to ask the patient to ensure a comprehensive diagnosis.
4. Offer classifications of possible diseases, including differential diagnoses (similar conditions to rule out).
5. Ask further clarifying questions that the doctor should consider, such as lifestyle factors, prior medical history, or exposure to local disease trends, to refine the diagnosis.

Please provide the necessary information, and I'll get started on assisting with diagnostic support.

### ❗️❗️ Make sure you clear GPU memory by clicking on Restart button above, if you want to use Streamlit from here

In [2]:
# check GPU usage

!nvidia-smi

Fri Jul  5 09:52:53 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.183.01             Driver Version: 535.183.01   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A10G                    Off | 00000000:00:1E.0 Off |                    0 |
|  0%   38C    P0              65W / 300W |   5956MiB / 23028MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--